# Matching

## Dependencies

In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

import sys
from pathlib import Path

# Automatically detect the repo root (parent of notebook folder)
repo_root = Path().resolve().parent  # if notebook is in 'notebooks/' folder
sys.path.append(str(repo_root))

from config.config import get_environment

from config.config import data_import_json, data_import_pandas, data_export_pandas

In [ ]:
import recordlinkage as rl
from recordlinkage.preprocessing import clean
from recordlinkage.compare import String#, CompareFunction

from rapidfuzz import fuzz

## ENV

In [ ]:
ENV = get_environment(
    env_path="../environments",
    env_name="env.json"
)

# content_date = datetime.now().date() + timedelta(days=0)
content_date = ENV['CONTENT_DATE']
version = ENV['VERSION']

website = 'all'
source = ENV['SOURCE']['NAME']
# target = ENV['TARGET']["1"]['NAME']
target = ENV['TARGET']["2"]['NAME']

## Matching

### Input

In [ ]:
df_input = data_import_pandas(
    website=website,
    content_date=content_date,
    version=version,
    folder_name=f'normalize/{website}',
    additional_info='normalize'
)

# Split source target
df_source = df_input[df_input['website'] == source].copy(deep=True).reset_index(drop=True)
df_target = df_input[df_input['website'] == target].copy(deep=True).reset_index(drop=True)

### Process Matching 1st Brand (Simple Computational Waste) - BACKUP

In [ ]:
# BACKUP
# # --- Preprocessing (Cleaning) ---
# df_source["clean_brand"] = clean(df_source["clean_brand"])
# df_target["clean_brand"] = clean(df_target["clean_brand"])

# df_source["clean_name"] = clean(df_source["clean_name"])
# df_target["clean_name"] = clean(df_target["clean_name"])

# threshold_brand = 0.6
# indexer = rl.Index()
# indexer.full() #.block('brand')

# pairs = indexer.index(df_source, df_target)


# # --- First-level comparer ---
# comparer = rl.Compare()
# comparer.add(
#     String(
#         "clean_brand",
#         "clean_brand",
#         method='cosine',   # broad filter
#         label="score_brand"
#     )
# )

# # Enable progress bar
# comparer.progress = True

# compare_result = comparer.compute(pairs, df_source, df_target)

# # flag = 1 if score ≥ threshold
# compare_result["flag_brand"] = (compare_result["score_brand"] >= threshold_brand).astype(int)

# # Filter to reduce computational process
# compare_result = compare_result[compare_result['score_brand'] > threshold_brand/2].reset_index(drop=False)

# compare_result.rename(columns={
#     'level_0': 'index_source',
#     'level_1': 'index_target'
# }, inplace=True)

# # Create pivot merge
# df_source_match = df_source.copy(deep=True)
# df_target_match = df_target.copy(deep=True)

# df_source_match['index_source'] = df_source_match.index
# df_target_match['index_target'] = df_target_match.index

# # Rename columns source with suffix
# exclude_cols = ['index_source']
# rename_cols = [col for col in df_source_match.columns if col not in exclude_cols]
# df_source_match.rename(columns={
#     col: f"{source}_{col}" for col in rename_cols
# }, inplace=True)

# # Rename columns target with suffix
# exclude_cols = ['index_target']
# rename_cols = [col for col in df_target_match.columns if col not in exclude_cols]
# df_target_match.rename(columns={
#     col: f"{target}_{col}" for col in rename_cols
# }, inplace=True)

# # Merge comparison result to the initial data
# df_match = pd.merge(
#     left=df_source_match,
#     right=compare_result,
#     on='index_source',
#     how='left'
# )

# # Merge safely (deduplicates target index)
# df_match = pd.merge(
#     left=df_match[df_match['index_target'].notna()],
#     right=df_target_match,
#     on='index_target',
#     how='left'
# )

### Process Matching 1st Brand (Complex Computational Saver)

In [ ]:
# Parameters
threshold_brand = 0.6  # similarity threshold
score_brand_col = f'{source}_{target}_score_brand'
flag_brand_col = f'{source}_{target}_flag_brand'
source_brand_col = f'{source}_clean_brand'
target_brand_col = f'{target}_clean_brand'
source_brand_index = f'{source}_index_brand'
target_brand_index = f'{target}_index_brand'

# Preprocessing (Cleaning)
df_source['clean_brand'] = clean(df_source['clean_brand'].fillna(''))
df_target['clean_brand'] = clean(df_target['clean_brand'].fillna(''))

# Create unique brand data
df_source_brand = df_source[['clean_brand']].drop_duplicates().reset_index(drop=True)
df_target_brand = df_target[['clean_brand']].drop_duplicates().reset_index(drop=True)

# Indexing
indexer = rl.Index()
indexer.full()  # For exact blocking, use indexer.block('clean_brand') if many exact matches
brand_pairs = indexer.index(df_source_brand, df_target_brand)

# Compare brands
comparer = rl.Compare()
comparer.add(
    String(
        'clean_brand',
        'clean_brand',
        method='cosine', # ['jaro', 'jarowinkler', 'levenshtein', 'damerau_levenshtein', 'qgram', 'cosine', 'smith_waterman', 'lcs']
        label=score_brand_col
    )
)
comparer.progress = True  # enable progress bar

compare_result_brand = comparer.compute(brand_pairs, df_source_brand, df_target_brand)

# Add flag based on threshold
compare_result_brand[flag_brand_col] = (compare_result_brand[score_brand_col] >= threshold_brand).astype(int)

# Map indices back to brand names
compare_result_brand = compare_result_brand.reset_index(drop=False)
compare_result_brand.rename(columns={
    'level_0': source_brand_index,
    'level_1': target_brand_index
}, inplace=True)

compare_result_brand[source_brand_col] = df_source_brand.loc[compare_result_brand[source_brand_index], 'clean_brand'].values
compare_result_brand[target_brand_col] = df_target_brand.loc[compare_result_brand[target_brand_index], 'clean_brand'].values

data_export_pandas(
    df_output=compare_result_brand,
    website=f'{source}_{target}',
    content_date=content_date,
    version=version,
    folder_name=f'matching/{website}',
    additional_info='matching_brand_unique',
    file_extension='xlsx'
)

# Pivot using the score
df_pivot_brand = compare_result_brand.pivot_table(
    index=source_brand_col,
    columns=target_brand_col,
    values=score_brand_col,
    fill_value=0  # fill missing comparisons with 0
)

# Flatten columns
df_pivot_brand = df_pivot_brand.reset_index().rename_axis(None, axis=1)

# Add top_score and top_match
# top_score
df_pivot_brand.insert(1, 'top_score', df_pivot_brand.drop(columns=[source_brand_col]).max(axis=1))

# top_match
df_pivot_brand.insert(1, 'top_match', df_pivot_brand.drop(columns=[source_brand_col, 'top_score']).idxmax(axis=1))

data_export_pandas(
    df_output=df_pivot_brand,
    website=f'{source}_{target}',
    content_date=content_date,
    version=version,
    folder_name=f'matching/{website}',
    additional_info='matching_brand_pivot',
    file_extension='xlsx'
)

# Filter only score > half of threshold
compare_result_brand = compare_result_brand[compare_result_brand[score_brand_col] > threshold_brand/2]

In [ ]:
# Merge back to full dataset
df_source_match = df_source.merge(
    compare_result_brand[[source_brand_index, target_brand_index, source_brand_col, target_brand_col, score_brand_col, flag_brand_col]],
    left_on='clean_brand',
    right_on=source_brand_col,
    how='left'
)

df_target_match = df_target.merge(
    compare_result_brand[~compare_result_brand[target_brand_index].duplicated()][[source_brand_index, target_brand_index, source_brand_col, target_brand_col, score_brand_col, flag_brand_col]],
    left_on='clean_brand',
    right_on=target_brand_col,
    how='left'
)

# Drop helper columns if desired
df_source_match.drop(columns=[source_brand_col, target_brand_col], inplace=True)
df_target_match.drop(columns=[source_brand_index, source_brand_col, target_brand_col, score_brand_col, flag_brand_col], inplace=True)

# Rename columns source with suffix
exclude_cols = [source_brand_index, target_brand_index, source_brand_col, target_brand_col, score_brand_col, flag_brand_col]
rename_cols = [col for col in df_source_match.columns if col not in exclude_cols]
df_source_match.rename(columns={
    col: f"{source}_{col}" for col in rename_cols
}, inplace=True)

# Rename columns target with suffix
exclude_cols = [source_brand_index, target_brand_index, source_brand_col, target_brand_col, score_brand_col, flag_brand_col]
rename_cols = [col for col in df_target_match.columns if col not in exclude_cols]
df_target_match.rename(columns={
    col: f"{target}_{col}" for col in rename_cols
}, inplace=True)

# Filter empty source_brand_index to avoid unnecessary empty merge explode
df_source_match = df_source_match[df_source_match[target_brand_index].notna()]
df_target_match = df_target_match[df_target_match[target_brand_index].notna()]

df_match = pd.merge(
    left=df_source_match,
    right=df_target_match,
    on=target_brand_index,
    how='left'
)


source_item_id = f'{source}_item_id'
target_item_id = f'{target}_item_id'

pair_id_col = f'{source}_{target}_id'

# Create pair id
df_match[pair_id_col] = df_match[[source_item_id, target_item_id]].apply(tuple, axis=1).str.join('---')

### Process Matching 2nd (String Name)

In [ ]:
threshold_name = 0.7
score_name_col = f'{source}_{target}_score_name'
flag_name_col = f'{source}_{target}_flag_name'
source_name_col = f'{source}_clean_name'
target_name_col = f'{target}_clean_name'

# Scoring name pair
from tqdm import tqdm
tqdm.pandas()

df_match[score_name_col] = df_match.progress_apply(
    lambda row: fuzz.token_set_ratio(
        row[source_name_col], 
        row[target_name_col]
    ) / 100,
    axis=1
)

# Create flag name
df_match[flag_name_col] = (df_match[score_name_col] >= threshold_name).astype(int)

### Process Matching 2nd (Embeddings Name) - BACKUP

In [ ]:
# BACKUP
# # Import embeddings name
# json_embed = data_import_json(
#     website=website,
#     content_date=content_date,
#     version=version,
#     folder_name=f'normalize/{website}',
#     additional_info='embeddings_name'
# )

# # Convert to pandas
# df_embed = pd.DataFrame(json_embed['data'])

In [ ]:
# # Merge back to df_match
# source_embed_col = f'{source}_embed_name'
# target_embed_col = f'{target}_embed_name'

# source_key_id = f'{source}_key_id'
# target_key_id = f'{target}_key_id'

# df_match[source_embed_col] = df_match[source_key_id].map(
#     df_embed.set_index('key_id')['embeddings_name'].to_dict()
# )

# df_match[target_embed_col] = df_match[target_key_id].map(
#     df_embed.set_index('key_id')['embeddings_name'].to_dict()
# )

# # Scoring
# from sklearn.metrics.pairwise import cosine_similarity

# # Custom cosine similarity function
# def cosine_score(pair):
#     x, y = pair
#     return cosine_similarity([x], [y])[0][0]

# def manhattan_score(pair):
#     x, y = pair
#     x = np.array(x)
#     y = np.array(y)
#     return 1 / (1 + np.sum(np.abs(x - y)))

# from numpy.linalg import norm
# def euclidean_score(pair):
#     x, y = pair
#     x = np.array(x)
#     y = np.array(y)
#     return 1 / (1 + norm(x - y))  # scale to (0, 1)

# df_match[score_name_col] = df_match[[source_embed_col, target_embed_col]].apply(tuple, axis=1).progress_apply(cosine_score)

# df_match.drop(columns=[source_embed_col, target_embed_col], inplace=True)

# # Create flag name
# df_match[flag_name_col] = (df_match[score_name_col] >= threshold_name).astype(int)

# # Create pair id
# df_match[pair_id_col] = df_match[[source_item_id, target_item_id]].apply(tuple, axis=1).str.join('---')

### Process Matching 3rd (Price Diff)

In [ ]:
threshold_price = 0.3 # percentage
score_price_col = f'{source}_{target}_score_price'
flag_price_col = f'{source}_{target}_flag_price'
source_price_col = f'{source}_clean_price'
target_price_col = f'{target}_clean_price'

flag_final_col = 'result'

# Scoring price diff pair in percentage
df_match[score_price_col] = abs(df_match[source_price_col] - df_match[target_price_col])/df_match[source_price_col]

# Create flag price
df_match[flag_price_col] = (df_match[score_price_col] <= threshold_price).astype(int)

### Result Matching

In [ ]:
rank_brand_col = f'{source}_{target}_rank_brand'
rank_name_col = f'{source}_{target}_rank_name'
rank_price_col = f'{source}_{target}_rank_price'

# Filter Necessary columns only
leading_cols = [f'{source}_{target}_id']
dynamic_cols = ['scrape_date', 'item_id', 'item_url', 'brand', 'item_name']
dynamic_cols = [f'{prefix}_{col}' for col in dynamic_cols for prefix in [source, target]]
trailing_cols = [score_brand_col, flag_brand_col, score_name_col, flag_name_col, score_price_col, flag_price_col]

df_match = df_match[leading_cols + dynamic_cols + trailing_cols].copy(deep=True)

# Filter only matched brand and matched name
df_match_candidate = df_match[
    (df_match[flag_brand_col] == 1)
    &
    (df_match[flag_name_col] == 1)
    &
    (df_match[flag_price_col] == 1)
].copy(deep=True)

df_match_candidate.sort_values(by=[score_brand_col, score_name_col], ascending=[False, False], inplace=True)

# Rank brand to handle multiple matches
df_match_candidate[rank_brand_col] = df_match_candidate.groupby(source_item_id)[score_brand_col].rank(
    method='dense',
    ascending=False
).astype(int)

# Rank name to handle multiple matches
df_match_candidate[rank_name_col] = df_match_candidate.groupby(source_item_id)[score_name_col].rank(
    method='dense',
    ascending=False
).astype(int)

# Rank name to handle multiple matches
df_match_candidate[rank_price_col] = df_match_candidate.groupby(source_item_id)[score_price_col].rank(
    method='dense',
    ascending=False
).astype(int)

df_match_candidate.sort_values(by=[rank_name_col, rank_price_col], ascending=[True, True], inplace=True)

# Final decision match/not match
df_match_candidate[flag_final_col] = np.where(
    (df_match_candidate[rank_name_col] == 1)
    &
    (~df_match_candidate[source_item_id].duplicated()),
    'match',
    'not match'
)

data_export_pandas(
    df_output=df_match_candidate,
    website=f'{source}_{target}',
    content_date=content_date,
    version=version,
    folder_name=f'matching/{website}',
    additional_info='matching_candidate',
    incl_excel=True
)

# Filter only for matched final result
df_match_matched = df_match_candidate[df_match_candidate[flag_final_col] == 'match'].copy(deep=True).reset_index(drop=True)

data_export_pandas(
    df_output=df_match_matched,
    website=f'{source}_{target}',
    content_date=content_date,
    version=version,
    folder_name=f'matching/{website}',
    additional_info='matching_matched',
    incl_excel=True
)

In [ ]:
# Merge back result to the complete pair
df_match[flag_final_col] = np.where(
    df_match[pair_id_col].isin(
        df_match_matched[pair_id_col]
    ),
    'match',
    'not match'
)

data_export_pandas(
    df_output=df_match,
    website=f'{source}_{target}',
    content_date=content_date,
    version=version,
    folder_name=f'matching/{website}',
    additional_info='matching_complete_pair'
)

In [ ]:
# len(df_match_matched[source_item_id].unique())/len(df_source)
# len(df_match_matched[target_item_id].unique())/len(df_target)

### Output